# 0. Set up

In [1]:
import os

wd_path = "/home/bastebun/work/project_paaral"
if os.getcwd() != wd_path:
    os.chdir(wd_path)
else:
    pass

In [2]:
import re
import sys
import copy
import math
import time
import pickle
import numpy as np
import pandas as pd
import multiprocessing
from collections import Counter
import matplotlib.pyplot as plt

from config.config import Config
cf = Config()

import importlib
def reload_module(module):
    importlib.reload(module)

# 1. Load data

## 1.1. Sequential

In [3]:
# !ls -lhsa "data/private/ESC/SY 2022-2023"

In [4]:
# dirpath_esc = "data/private/ESC"
# esc_yrs = os.listdir(dirpath_esc)
# print(esc_yrs)

In [5]:
# header, np_vals = 0, []
# # for i, yr in enumerate(esc_yrs):
# for i, yr in enumerate(["SY 2023-2024"]):
#     print(f"At school year {yr}")
#     dirpath = os.path.join(dirpath_esc, yr)
#     excel_fnames = os.listdir(dirpath)
#     # print(excel_fnames)

#     for fname in excel_fnames:
#         print(f"Processing sheet {fname}")
#         fpath = os.path.join(dirpath, fname)
#         sheet_name = re.findall(r'(.*?).xlsx', fname)[0]
#         df_tmp = pd.read_excel(fpath, sheet_name=sheet_name)
#         df_tmp = df_tmp.drop(columns=['Full Name','Encoded Upper Grade Level'])
#         df_tmp['sheet_name'] = sheet_name
#         df_tmp['school_year'] = yr
#         df_tmp.columns = [str(col).lower() for col in df_tmp.columns]
#         # print(df_tmp.shape)
        
#         np_val = df_tmp.to_numpy()
#         np_vals.append(np_val)

#         if i == 0:
#             header = df_tmp.columns
#         print(f"Done processing sheet {fname}. Shape {df_tmp.shape}")
#     # break

In [6]:
# # Save a snapshot of ESC beneficiaries for one school year
# path = "output/esc_beneficiaries_sy2324.csv"
# df_yr.to_csv(path, index=False)

In [7]:
# try:
#     df_yr = pd.DataFrame(data=np.vstack(np_vals), columns=header)
#     print(df_yr.shape)
# except:
#     path = "output/esc_beneficiaries_sy2324.csv"
#     df_yr = pd.read_csv(path)
#     print(df_yr.shape)

# reg_agg = df_yr.groupby(['sheet_name'])['lrn'].nunique().sort_values(ascending=False).to_frame()
# display(reg_agg.style.format(thousands=',', precision=0))

## 1.2. Parallelized

In [8]:
dirpath_esc = "data/private/ESC"
esc_yrs = os.listdir(dirpath_esc)
print(esc_yrs)

['SY 2022-2023', 'SY 2023-2024', 'SY 2024-2025']


In [9]:
def process_excel_file(file_info):
    """
    A worker function to process a single Excel file.
    This function will be run in a separate process.
    """
    dirpath, fname, yr = file_info
    print(f"Processing sheet {fname}")
    
    try:
        fpath = os.path.join(dirpath, fname)
        sheet_name = re.findall(r'(.*?).xlsx', fname)[0]
        df_tmp = pd.read_excel(fpath, sheet_name=sheet_name)
        df_tmp = df_tmp.drop(columns=['Full Name', 'Encoded Upper Grade Level'])
        df_tmp['sheet_name'] = sheet_name
        df_tmp['school_year'] = yr
        df_tmp.columns = [str(col).lower() for col in df_tmp.columns]
        
        np_val = df_tmp.to_numpy()
        header = df_tmp.columns.tolist()  # Convert to list for pickling
        
        print(f"Done processing sheet {fname}. Shape {df_tmp.shape}")
        
        # Return both the NumPy array and the header
        return np_val, header
    except Exception as e:
        print(f"Error processing file {fname}: {e}")
        return None, None

In [10]:
# Get the number of CPU cores available
num_cores = multiprocessing.cpu_count()
print(f"Using {num_cores} cores for parallel processing.")

Using 10 cores for parallel processing.


In [11]:
s_time = time.time()

header = None
np_vals = []

path_output = "output"
# Create a pool of worker processes
with multiprocessing.Pool(processes=num_cores) as pool:
    for i, yr in enumerate(esc_yrs):
        print(f"At school year {yr}")
        dirpath = os.path.join(dirpath_esc, yr)
        excel_fnames = os.listdir(dirpath)
        
        # Create a list of tuples (dirpath, fname) to pass to the worker function
        tasks = [(dirpath, fname, yr) for fname in excel_fnames]
        
        # Map the `process_excel_file` function to the list of tasks
        # This is where the parallelization happens
        results = pool.map(process_excel_file, tasks)
        
        # Process the results from the parallel tasks
        for np_val, headers in results:
            if np_val is not None:
                np_vals.append(np_val)
                if header is None:
                    header = headers

        savepath = os.path.join(path_output, f"esc_{yr}.pkl")
        print(f"Saving current numpy values with path {savepath}")
        with open(savepath, 'wb') as file:
            pickle.dump(np_vals, file)

# After the pool closes, you can work with the collected data
print("All files processed.")
print(f"Total NumPy arrays collected: {len(np_vals)}")
print(f"Final header: {header}\n")

e_time = time.time() - s_time
print(f"Time elapsed: {(e_time / 60):.2f} minutes")

Processing sheet BARMM.xlsxProcessing sheet CAR.xlsxProcessing sheet NCR.xlsx
Processing sheet R02.xlsxProcessing sheet R03.xlsxProcessing sheet R01.xlsxProcessing sheet R06.xlsxProcessing sheet R05.xlsxProcessing sheet R04-B.xlsxProcessing sheet R04-A.xlsx








At school year SY 2022-2023
Done processing sheet R04-B.xlsx. Shape (23684, 9)
Processing sheet R07.xlsx
Done processing sheet CAR.xlsx. Shape (25147, 9)
Processing sheet R08.xlsx
Done processing sheet BARMM.xlsx. Shape (29854, 9)
Processing sheet R09.xlsx
Done processing sheet R02.xlsx. Shape (34708, 9)
Processing sheet R10.xlsx
Done processing sheet R01.xlsx. Shape (46636, 9)
Processing sheet R11.xlsx
Done processing sheet R05.xlsx. Shape (47127, 9)
Processing sheet R12.xlsx
Done processing sheet R08.xlsx. Shape (24456, 9)
Processing sheet R13.xlsx
Done processing sheet R09.xlsx. Shape (22577, 9)
Done processing sheet R06.xlsx. Shape (61589, 9)
Done processing sheet R13.xlsx. Shape (23997, 9)
Done processing sheet R12.xlsx

In [12]:
len(header)

9

In [13]:
np.vstack(np_vals)

array([[475511, 133526130177, 1603520, ..., 9000.0, 'BARMM',
        'SY 2022-2023'],
       [475511, 133536150012, 1603520, ..., 9000.0, 'BARMM',
        'SY 2022-2023'],
       [475511, 133536150011, 1603520, ..., 9000.0, 'BARMM',
        'SY 2022-2023'],
       ...,
       [406039, 132834150022, 1301787, ..., 9000, 'R13', 'SY 2024-2025'],
       [406039, 115224140076, 1301787, ..., 9000, 'R13', 'SY 2024-2025'],
       [406039, 132835140095, 1301787, ..., 9000, 'R13', 'SY 2024-2025']],
      shape=(2681668, 9), dtype=object)

# 2. Preprocess
* * *
The DepEd LRN is comprised of 12 digits, which is broken down into to the following components:
- First six (6) digits: School ID of school when registered
- Next two (2) digits: Year when learner is registered
- Last four (4) digits: Unique learner number

In [14]:
s_time = time.time()

esc_beneficiaries = pd.DataFrame(
    data=np.vstack(np_vals),
    columns=header
)
esc_beneficiaries = esc_beneficiaries.reset_index(drop=True)

# Streamline column names
replace_cols = {
    'deped id':'deped_school_id',
    'school id':'esc_school_id',
    'school name':'school_name',
    'billing statement no.':'billing_statement_number',
    'amount':'esc_subsidy_amount',
}
esc_beneficiaries = esc_beneficiaries.rename(
    columns=replace_cols
)

# Relabel region names to match with LIS region names
esc_regions = esc_beneficiaries['sheet_name'].unique().tolist()
regions_depet_lst = [
    'BARMM','CAR','NCR','Region I','Region II','Region III',
    'Region IV-A','MIMAROPA','Region V','Region VI','Region VII',
    'Region VIII','Region IX','Region X','Region XI','Region XII',
    'CARAGA'
]
# print(len(regions_depet_lst))

regions_enc = {k:v for k,v in zip(esc_regions, regions_depet_lst)}
# display(regions_enc)
esc_beneficiaries['region'] = esc_beneficiaries['sheet_name'].replace(regions_enc)

# Define data types before saving
df_dtypes = {
    'deped_school_id':'string',
    'lrn':'string',
    'esc_school_id':'string',
    'school_name':'string',
    'grade':'string',
    'billing_statement_number':'string',
    'esc_subsidy_amount':'float',
    'sheet_name':'category',
    'school_year':'category',
    'region':'category',
}
esc_beneficiaries = esc_beneficiaries.astype(df_dtypes)

# Create new column for complete grade level
esc_beneficiaries['grade_level'] = "Grade "+ esc_beneficiaries['grade']
esc_beneficiaries['grade_level'] = esc_beneficiaries['grade_level'].astype('category')

def get_lrn(val):
    try:
        if "." in val:
            return re.findall(r'(\d+)\.', val)[0]
        elif len(val) < 12:
            return "invalid_lrn_lt12"
        elif len(val) > 12:
            return "invalid_lrn_gt12"
        else:
            return val
    except: # for NaN types
        return "no_lrn"

# There are LRNs in SY 23-24 that have decimal places. We addresss this.
esc_beneficiaries['lrn_validated'] = esc_beneficiaries['lrn'].apply(get_lrn)
esc_beneficiaries['lrn_validated'] = esc_beneficiaries['lrn_validated'].astype('string')

# Extract school ID from validated LRNs and assign new column
def get_school_id_from_valid_lrn(val):
    try:
        if len(val) == 12 or str(val) not in ['invalid_lrn_lt12']:
            # Get first six digits whether validated LRN is 12 digits or more
            return re.findall(r'(^\d{6})', val)[0]
        else:
            return np.nan
    except:
        return np.nan
    
esc_beneficiaries['lrn_school_id'] = esc_beneficiaries['lrn_validated'].apply(get_school_id_from_valid_lrn)
# esc_beneficiaries['lrn_school_id'] = esc_beneficiaries['lrn_school_id'].astype('string')

# Log total runtime of preprocessing
e_time = time.time() - s_time
print(f"Time elapsed: {e_time:.2f} seconds\n")

print(esc_beneficiaries.shape)
display(esc_beneficiaries.head(3))

Time elapsed: 82.37 seconds

(2681668, 13)


,deped_school_id,lrn,esc_school_id,school_name,grade,billing_statement_number,esc_subsidy_amount,sheet_name,school_year,region,grade_level,lrn_validated,lrn_school_id
0,475511,133526130177,1603520,"Adiong Memorial College Foundation, Inc.",9,ESC-227153,9000.0,BARMM,SY 2022-2023,BARMM,Grade 9,133526130177,133526
1,475511,133536150012,1603520,"Adiong Memorial College Foundation, Inc.",7,ESC-227153,9000.0,BARMM,SY 2022-2023,BARMM,Grade 7,133536150012,133536
2,475511,133536150011,1603520,"Adiong Memorial College Foundation, Inc.",7,ESC-227153,9000.0,BARMM,SY 2022-2023,BARMM,Grade 7,133536150011,133536


In [15]:
# display(esc_beneficiaries.tail(3))

In [16]:
# Check for NaNs
display(esc_beneficiaries.isna().sum())

deped_school_id                0
lrn                            6
esc_school_id                  0
school_name                    0
grade                          0
billing_statement_number       0
esc_subsidy_amount             0
sheet_name                     0
school_year                    0
region                         0
grade_level                    0
lrn_validated                  0
lrn_school_id               1629
dtype: int64

In [17]:
# Display a sample of rows whose LRNs are NaNs or invalid (or < 12 digits)
lrn_na = esc_beneficiaries[esc_beneficiaries['lrn_school_id'].isna()]
print(lrn_na.shape)
display(lrn_na.head())

(1629, 13)


,deped_school_id,lrn,esc_school_id,school_name,grade,billing_statement_number,esc_subsidy_amount,sheet_name,school_year,region,grade_level,lrn_validated,lrn_school_id
7,475511,10059150018,1603520,"Adiong Memorial College Foundation, Inc.",9,ESC-227153,9000.0,BARMM,SY 2022-2023,BARMM,Grade 9,invalid_lrn_lt12,NaN
8,475511,10059150017,1603520,"Adiong Memorial College Foundation, Inc.",9,ESC-227153,9000.0,BARMM,SY 2022-2023,BARMM,Grade 9,invalid_lrn_lt12,NaN
10,475511,10378150014,1603520,"Adiong Memorial College Foundation, Inc.",10,ESC-227153,9000.0,BARMM,SY 2022-2023,BARMM,Grade 10,invalid_lrn_lt12,NaN
336,475511,11050160005,1603520,"Adiong Memorial College Foundation, Inc.",9,ESC-227153,9000.0,BARMM,SY 2022-2023,BARMM,Grade 9,invalid_lrn_lt12,NaN
969,405797,11240110357,1201705,"Antonio R. Pacheco College, Inc.",10,ESC-224610,9000.0,BARMM,SY 2022-2023,BARMM,Grade 10,invalid_lrn_lt12,NaN


In [18]:
# We use the raw extracted data to review invalid LRNs
esc_bf = pd.DataFrame(
    data=np.vstack(np_vals),
    columns=header
)
esc_bf['lrn'] = esc_bf['lrn'].astype('string')

display(esc_bf.loc[[665928,776789,969,2677262,910784], :])

,deped id,lrn,school id,school name,grade,billing statement no.,amount,sheet_name,school_year
665928,404248,20199,701108,"St. PaulÃ¢â‚¬â„¢s Academy of Inabanga, Bohol, ...",10,ESC-223488,9000.0,R07,SY 2022-2023
776789,400793,1267,1001405,"Valencia Baptist Christian Academy, Inc.",10,ESC-223550,9000.0,R10,SY 2022-2023
969,405797,11240110357,1201705,"Antonio R. Pacheco College, Inc.",10,ESC-224610,9000.0,BARMM,SY 2022-2023
2677262,406039,2350020029,1301787,Southern Technological Institute of the Philip...,9,ESC-278148,9000,R13,SY 2024-2025
910784,405953,13186140109,1301749,"Father Saturnino Urios College of Sibagat, Inc.",10,ESC-229481,9000,R13,SY 2022-2023


In [19]:
# Check the string length of the extracted school IDs
Counter([len(str(el)) for el in esc_bf['lrn'].to_list()])

Counter({12: 2393243,
         14: 286434,
         11: 1538,
         13: 303,
         10: 107,
         9: 14,
         8: 13,
         6: 8,
         4: 7,
         5: 1})

In [20]:
# Review data types
esc_beneficiaries.dtypes

deped_school_id             string[python]
lrn                         string[python]
esc_school_id               string[python]
school_name                 string[python]
grade                       string[python]
billing_statement_number    string[python]
esc_subsidy_amount                 float64
sheet_name                        category
school_year                       category
region                            category
grade_level                       category
lrn_validated               string[python]
lrn_school_id                       object
dtype: object

## 2.1 Save as parquet

In [21]:
# !ls "data/processed"

In [22]:
!rm -rf "data/processed/esc_beneficiaries.parquet"

# Save as parquet
path_parquet = "data/processed/esc_beneficiaries.parquet"
esc_beneficiaries.to_parquet(
    path_parquet,
    partition_cols='region',
)

## 2.2 Load parquet

In [23]:
%%time
# Load parquet
path_parquet = "data/processed/esc_beneficiaries.parquet"
df_esc = pd.read_parquet(path_parquet)

print(df_esc.shape)
display(df_esc.head(3))

(2681668, 13)


,deped_school_id,lrn,esc_school_id,school_name,grade,billing_statement_number,esc_subsidy_amount,sheet_name,school_year,grade_level,lrn_validated,lrn_school_id,region
0,475511,133526130177,1603520,"Adiong Memorial College Foundation, Inc.",9,ESC-227153,9000.0,BARMM,SY 2022-2023,Grade 9,133526130177,133526,BARMM
1,475511,133536150012,1603520,"Adiong Memorial College Foundation, Inc.",7,ESC-227153,9000.0,BARMM,SY 2022-2023,Grade 7,133536150012,133536,BARMM
2,475511,133536150011,1603520,"Adiong Memorial College Foundation, Inc.",7,ESC-227153,9000.0,BARMM,SY 2022-2023,Grade 7,133536150011,133536,BARMM


CPU times: user 22.7 s, sys: 8.37 s, total: 31.1 s
Wall time: 15.8 s


In [24]:
display(df_esc.dtypes)

deped_school_id             string[python]
lrn                         string[python]
esc_school_id               string[python]
school_name                 string[python]
grade                       string[python]
billing_statement_number    string[python]
esc_subsidy_amount                 float64
sheet_name                        category
school_year                       category
grade_level                       category
lrn_validated               string[python]
lrn_school_id                       object
region                            category
dtype: object

In [25]:
%%time
df_agg = df_esc.groupby(['school_year']).agg(
    {
        'lrn':'nunique',
        'deped_school_id':'nunique',
        'esc_subsidy_amount':'sum',
    }
)
display(df_agg.style.format(thousands=',', precision=0))

<timed exec>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


,lrn,deped_school_id,esc_subsidy_amount
school_year,,,
SY 2022-2023,"920,875","3,561","8,843,860,587"
SY 2023-2024,"897,015","3,592","8,623,657,566"
SY 2024-2025,"863,753","3,533","8,309,483,105"


CPU times: user 9.38 s, sys: 410 ms, total: 9.79 s
Wall time: 9.82 s


In [38]:
df_esc[df_esc['school_year'] == "SY 2023-2024"].groupby(['region'], observed=True)['lrn'].nunique().sort_values(ascending=False)

region
Region IV-A    163832
Region III     119472
NCR             96007
Region VII      68815
Region VI       58777
Region X        49272
Region V        45053
Region I        44059
Region XI       37605
Region XII      35175
Region II       33320
BARMM           30328
CAR             24161
Region VIII     23434
CARAGA          23377
MIMAROPA        22819
Region IX       21509
Name: lrn, dtype: int64